<a href="https://colab.research.google.com/github/jmhuer/quantum_dots/blob/main/quantum_dots_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load data from excell


In [3]:
!pip install skorch
!pip install tqdm


In [4]:
import pandas as pd

def open_excell(filename):
    excell = pd.ExcelFile(filename)
    excell.sheet_names
    df = excell.parse("Sheet1")
    df.columns = df.columns.map(str)
    df = df.dropna().reset_index(drop=True)
    return df

df = open_excell("/content/QDots data - Juan.xlsx")

## Pre-processing + model defenitions before training

In [5]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

X_columns  = ["415","445","480", "515", "555", "590", "630", "680"]
Y1_columns = ["# emitters λ1 (x1010)"]
Y2_columns = ["# emitters λ2 (x1010)"]
Y_columns  = ["# emitters λ1 (x1010)", "# emitters λ2 (x1010)"]

##LINEAR REGRESSION
lr_reg = make_pipeline(LinearRegression())

##RandomForestRegressor
rf_reg = make_pipeline(StandardScaler(), RandomForestRegressor())
parameters = {'randomforestregressor__max_depth':[1,2,3,4,5]}
rf_reg = GridSearchCV(rf_reg, parameters)
  
##SVM
svr = make_pipeline(StandardScaler(), SVR(kernel='linear'))
parameters = {'svr__kernel':['linear', 'rbf'], 'svr__epsilon':[0.1, 0.2, 0.3,0.6], 'svr__C': [0.1,0.5,1,2,3]}
svr = GridSearchCV(svr, parameters)


# NN stuff here

In [6]:
from torch import nn
from skorch import NeuralNetRegressor

class MultivariateLinearRegression(nn.Module):
    def __init__(self, input = 8, num_units=10, nonlin=nn.LeakyReLU()):
          super(MultivariateLinearRegression, self).__init__()
          self.dense0 = nn.Linear(input, num_units)
          self.nonlin = nonlin
          self.dropout = nn.Dropout(0.1)
          self.output = nn.Linear(num_units, 1)
    def forward(self, X, **kwargs):
          X = self.nonlin(self.dense0(X))
          X = self.dropout(X)
          X = self.output(X)
          return X

net = NeuralNetRegressor(
    MultivariateLinearRegression().double(),
    iterator_train__shuffle = False,
    train_split = False,
    verbose = 0)

myNN_reg = make_pipeline(StandardScaler(), net)

params = {
    'neuralnetregressor__lr': [0.001, 0.002, 0.003],
    'neuralnetregressor__max_epochs': [10, 20, 30]
}
myNN_reg = GridSearchCV(myNN_reg, params)


None


##Training loop w/ cross-validation 

In [7]:
from tqdm import tqdm

##For Cross validations 
kf = KFold(n_splits=10, random_state=42, shuffle=True) # Define the split - into 10 folds 
k_accuracy = {"R2_lr": [],"MSE_lr": [], "R2_rf": [],"MSE_rf": [],"R2_svr": [],"MSE_svr": [],"R2_nn": [],"MSE_nn": []}

##a progress bar because Im fancy
pbar = tqdm(total=100,position=0, leave=True)

##MAIN LOOP
for train_index, test_index in kf.split(df):
    X_train, X_test = df.loc[train_index, X_columns], df.loc[test_index, X_columns]
    Y_train, Y_test = df.loc[train_index, Y1_columns], df.loc[test_index, Y1_columns]

    ##LINEAR REGRESSION
    lr_reg.fit(X_train, Y_train.values.ravel())
    Y_pred = lr_reg.predict(X_test)
    #store preformance metrics
    k_accuracy['MSE_lr'].append(mean_squared_error(Y_test, Y_pred))
    k_accuracy['R2_lr'].append(lr_reg.score(X_test, Y_pred))


    ##RandomForestRegressor
    rf_reg.fit(X_train, Y_train.values.ravel())
    Y_pred = rf_reg.predict(X_test)
    #store preformance metrics
    k_accuracy['R2_rf'].append(mean_squared_error(Y_test, Y_pred))
    k_accuracy['MSE_rf'].append(rf_reg.score(X_test, Y_pred))


    ##Support Vecotor Regressor 
    svr.fit(X_train, Y_train.values.ravel())
    Y_pred = svr.predict(X_test)
    #store preformance metrics
    k_accuracy['MSE_svr'].append(mean_squared_error(Y_test, Y_pred))
    k_accuracy['R2_svr'].append(svr.score(X_test, Y_pred))


    ##NN Regressor
    myNN_reg.fit(X_train, Y_train.values)
    Y_pred = myNN_reg.predict(X_test)
    #store preformance metrics
    k_accuracy['MSE_nn'].append(mean_squared_error(Y_test, Y_pred))
    k_accuracy['R2_nn'].append(myNN_reg.score(X_test, Y_pred))

    pbar.update(10)
pbar.close()
print('\n')

for i in k_accuracy.keys():
    print('\n', i," : ", sum(k_accuracy[i]) / len(k_accuracy[i]))
    print("---")

100%|██████████| 100/100 [01:42<00:00,  1.02s/it]




 R2_lr  :  1.0
---

 MSE_lr  :  4.050225300323952
---

 R2_rf  :  0.7954854691876657
---

 MSE_rf  :  1.0
---

 R2_svr  :  1.0
---

 MSE_svr  :  4.476981793012247
---

 R2_nn  :  1.0
---

 MSE_nn  :  13.097932144468876
---


# Ploting utils

In [8]:
import plotly.graph_objects as graph
def plot(all_history:list, title:str, log = False):
    """
    input:
        all_history: list of dicts to plot
    ret:
        None: show plotly fig
    """
    fig = graph.Figure(layout = graph.Layout(title=graph.layout.Title(text=title))) 
    for i in range(len(all_history)):
        fig.add_trace(graph.Scatter(x = all_history[i]["x"], 
                                    y = all_history[i]["y"],
                                    name = all_history[i]["legend"])) 
    if log: fig.update_xaxes(type="log")
    fig.show()
